# Final Capstone for IBM Course

In [193]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
from bs4 import BeautifulSoup
import re

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans
import folium # plotting library
import matplotlib.cm as cm
import matplotlib.colors as colors

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


## Transforming the Wikipedia page into a Beautiful Soup

In [111]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
r = requests.get(url)
soup = BeautifulSoup(r.content)

In [109]:
#Created a sub soup where everything started with tr
title = soup.find_all('tr')
#Initialize the iterations
i=0
dict ={}
list1 =[]
list2 =[]
list3 =[]
for element in title:
    #Iterate through the sub soup
    i = i+1
    #Created lists with only the post code, the borough and the neighborhood
    n =element.get_text().split('\n')
    n = [string for string in n if string != ""]
    #Defined a dictionary with the respective header
    if i==1:
        dict[n[0]] = list1
        dict[n[1]] = list2
        dict[n[2]] = list3
    #Excluded non assigned terms
    elif 'Not assigned' not in n:
        list1.append(n[0])
        list2.append(n[1])
        list3.append(n[2])
    #Ended in the last needed term
    if i ==181:
        break

In [110]:
#Created a Pandas DataFrame with the dictionary
df = pd.DataFrame.from_dict(dict)
pd.set_option('display.max_rows', 1000)
df.shape

(103, 3)

# Inserting Latitude and Longitude

In [123]:
df1 = pd.read_csv('geo.csv')

In [ ]:
#Renaming to merge the dataframes
df.rename(columns={'Postal code':'Postal Code'}, inplace =True)

In [132]:
dff = pd.merge(df, df1, on="Postal Code")

In [133]:
dff

Postal Code           Borough  \
0           M3A        North York   
1           M4A        North York   
2           M5A  Downtown Toronto   
3           M6A        North York   
4           M7A  Downtown Toronto   
5           M9A         Etobicoke   
6           M1B       Scarborough   
7           M3B        North York   
8           M4B         East York   
9           M5B  Downtown Toronto   
10          M6B        North York   
11          M9B         Etobicoke   
12          M1C       Scarborough   
13          M3C        North York   
14          M4C         East York   
15          M5C  Downtown Toronto   
16          M6C              York   
17          M9C         Etobicoke   
18          M1E       Scarborough   
19          M4E      East Toronto   
20          M5E  Downtown Toronto   
21          M6E              York   
22          M1G       Scarborough   
23          M4G         East York   
24          M5G  Downtown Toronto   
25          M6G  Downtown Toronto   
26          M1H       Scarborough   
27          M2H        North York   
28          M3H        North York   
29          M4H         East York   
30          M5H  Downtown Toronto   
31          M6H      West Toronto   
32          M1J       Scarborough   
33          M2J        North York   
34          M3J        North York   
35          M4J         East York   
36          M5J  Downtown Toronto   
37          M6J      West Toronto   
38          M1K       Scarborough   
39          M2K        North York   
40          M3K        North York   
41          M4K      East Toronto   
42          M5K  Downtown Toronto   
43          M6K      West Toronto   
44          M1L       Scarborough   
45          M2L        North York   
46          M3L        North York   
47          M4L      East Toronto   
48          M5L  Downtown Toronto   
49          M6L        North York   
50          M9L        North York   
51          M1M       Scarborough   
52          M2M        North York   
53          M3M        North York   
54          M4M      East Toronto   
55          M5M        North York   
56          M6M              York   
57          M9M        North York   
58          M1N       Scarborough   
59          M2N        North York   
60          M3N        North York   
61          M4N   Central Toronto   
62          M5N   Central Toronto   
63          M6N              York   
64          M9N              York   
65          M1P       Scarborough   
66          M2P        North York   
67          M4P   Central Toronto   
68          M5P   Central Toronto   
69          M6P      West Toronto   
70          M9P         Etobicoke   
71          M1R       Scarborough   
72          M2R        North York   
73          M4R   Central Toronto   
74          M5R   Central Toronto   
75          M6R      West Toronto   
76          M7R       Mississauga   
77          M9R         Etobicoke   
78          M1S       Scarborough   
79          M4S   Central Toronto   
80          M5S  Downtown Toronto   
81          M6S      West Toronto   
82          M1T       Scarborough   
83          M4T   Central Toronto   
84          M5T  Downtown Toronto   
85          M1V       Scarborough   
86          M4V   Central Toronto   
87          M5V  Downtown Toronto   
88          M8V         Etobicoke   
89          M9V         Etobicoke   
90          M1W       Scarborough   
91          M4W  Downtown Toronto   
92          M5W  Downtown Toronto   
93          M8W         Etobicoke   
94          M9W         Etobicoke   
95          M1X       Scarborough   
96          M4X  Downtown Toronto   
97          M5X  Downtown Toronto   
98          M8X         Etobicoke   
99          M4Y  Downtown Toronto   
100         M7Y      East Toronto   
101         M8Y         Etobicoke   
102         M8Z         Etobicoke   

                                          Neighborhood   Latitude  Longitude  
0                                            Parkwoods  43.753259 -79.329656

# Analysis

In [142]:
print('Boroughs:',len(dff['Borough'].unique()),'Neighborhoods:', len(dff['Neighborhood'].unique()))

Boroughs: 10 Neighborhoods: 98


In [143]:
address = 'Toronto, CA'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

In [144]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
for lat, lng, borough, neighborhood in zip(dff['Latitude'],dff['Longitude'], dff['Borough'], dff['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='orange',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [147]:

from IPython.display import HTML
from IPython.display import display

# Taken from https://stackoverflow.com/questions/31517194/how-to-hide-one-specific-cell-input-or-output-in-ipython-notebook
tag = HTML('''<script>
code_show=true; 
function code_toggle() {
    if (code_show){
        $('div.cell.code_cell.rendered.selected div.input').hide();
    } else {
        $('div.cell.code_cell.rendered.selected div.input').show();
    }
    code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
To show/hide this cell's raw code input, click <a href="javascript:code_toggle()">here</a>.''')
display(tag)

CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30

In [146]:
neighborhood_latitude = dff.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude =dff.loc[0, 'Longitude'] # neighborhood longitude value
neighborhood_name = dff.loc[0, 'Neighborhood'] # neighborhood name

In [162]:
#Getting the venues nearby a point
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [163]:
#Retrieving the Venues in Toronto
toronto_venues = getNearbyVenues(names=dff['Neighborhood'],
                                   latitudes=dff['Latitude'],
                                   longitudes=dff['Longitude'])

Parkwoods
Victoria Village
Regent Park / Harbourfront
Lawrence Manor / Lawrence Heights
Queen's Park / Ontario Provincial Government
Islington Avenue
Malvern / Rouge
Don Mills
Parkview Hill / Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park / Princess Gardens / Martin Grove / Islington / Cloverdale
Rouge Hill / Port Union / Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate / Bloordale Gardens / Old Burnhamthorpe / Markland Wood
Guildwood / Morningside / West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor / Wilson Heights / Downsview North
Thorncliffe Park
Richmond / Adelaide / King
Dufferin / Dovercourt Village
Scarborough Village
Fairview / Henry Farm / Oriole
Northwood Park / York University
East Toronto
Harbourfront East / Union Station / Toronto Islands
Little Portugal / Trinity
Kennedy Park / Ionview / East Birchmount Park
Bayview Village
Do

In [164]:
toronto_venues

Neighborhood  \
0                                             Parkwoods   
1                                             Parkwoods   
2                                      Victoria Village   
3                                      Victoria Village   
4                                      Victoria Village   
...                                                 ...   
1670  Mimico NW / The Queensway West / South of Bloo...   
1671  Mimico NW / The Queensway West / South of Bloo...   
1672  Mimico NW / The Queensway West / South of Bloo...   
1673  Mimico NW / The Queensway West / South of Bloo...   
1674  Mimico NW / The Queensway West / South of Bloo...   

      Neighborhood Latitude  Neighborhood Longitude                   Venue  \
0                 43.753259              -79.329656         Brookbanks Park   
1                 43.753259              -79.329656           Variety Store   
2                 43.725882              -79.315572  Victoria Village Arena   
3                 43.725882              -79.315572             Tim Hortons   
4                 43.725882              -79.315572               Portugril   
...                     ...                     ...                     ...   
1670              43.628841              -79.520999                    RONA   
1671              43.628841              -79.520999       Once Upon A Child   
1672              43.628841              -79.520999           Value Village   
1673              43.628841              -79.520999    Kingsway Boxing Club   
1674              43.628841              -79.520999            Burrito Boyz   

      Venue Latitude  Venue Longitude          Venue Category  
0          43.751976       -79.332140                    Park  
1          43.751974       -79.333114       Food & Drink Shop  
2          43.723481       -79.315635            Hockey Arena  
3          43.725517       -79.313103             Coffee Shop  
4          43.725819       -79.312785   Portuguese Restaurant  
...              ...              ...                     ...  
1670       43.629393       -79.518320          Hardware Store  
1671       43.631075       -79.518290              Kids Store  
1672       43.631269       -79.518238  Thrift / Vintage Store  
1673       43.627254       -79.526684                     Gym  
1674       43.626657       -79.526349           Burrito Place  

[1675 rows x 7 columns]

In [177]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

In [178]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

Neighborhood  Accessories Store  \
0                                           Agincourt           0.000000   
1                             Alderwood / Long Branch           0.000000   
2   Bathurst Manor / Wilson Heights / Downsview North           0.000000   
3                                     Bayview Village           0.000000   
4                  Bedford Park / Lawrence Manor East           0.000000   
5                                         Berczy Park           0.000000   
6                        Birch Cliff / Cliffside West           0.000000   
7      Brockton / Parkdale Village / Exhibition Place           0.000000   
8               Business reply mail Processing CentrE           0.000000   
9   CN Tower / King and Spadina / Railway Lands / ...           0.000000   
10                                Caledonia-Fairbanks           0.000000   
11              Canada Post Gateway Processing Centre           0.000000   
12                                          Cedarbrae           0.000000   
13                                 Central Bay Street           0.000000   
14                                           Christie           0.000000   
15                               Church and Wellesley           0.000000   
16          Clarks Corners / Tam O'Shanter / Sullivan           0.000000   
17  Cliffside / Cliffcrest / Scarborough Village West           0.000000   
18                    Commerce Court / Victoria Hotel           0.000000   
19                                         Davisville           0.000000   
20                                   Davisville North           0.000000   
21  Del Ray / Mount Dennis / Keelsdale and Silvert...           0.000000   
22                                          Don Mills           0.000000   
23  Dorset Park / Wexford Heights / Scarborough To...           0.000000   
24                                          Downsview           0.000000   
25                      Dufferin / Dovercourt Village           0.000000   
26                                       East Toronto           0.000000   
27  Eringate / Bloordale Gardens / Old Burnhamthor...           0.000000   
28                     Fairview / Henry Farm / Oriole           0.020000   
29            First Canadian Place / Underground city           0.000000   
30                           Forest Hill North & West           0.000000   
31                           Garden District, Ryerson           0.000000   
32                                          Glencairn           0.000000   
33                  Golden Mile / Clairlea / Oakridge           0.000000   
34                Guildwood / Morningside / West Hill           0.000000   
35  Harbourfront East / Union Station / Toronto Is...           0.000000   
36                     High Park / The Junction South           0.000000   
37                                  Hillcrest Village           0.000000   
38                                      Humber Summit           0.000000   
39                                  Humberlea / Emery           0.000000   
40                                 Humewood-Cedarvale           0.000000   
41                    India Bazaar / The Beaches West           0.000000   
42      Kennedy Park / Ionview / East Birchmount Park           0.000000   
43        Kensington Market / Chinatown / Grange Park           0.000000   
44  Kingsview Village / St. Phillips / Martin Grov...           0.000000   
45                  Lawrence Manor / Lawrence Heights           0.071429   
46                                      Lawrence Park           0.000000   
47                                            Leaside           0.000000   
48                          Little Portugal / Trinity           0.000000   
49                                    Malvern / Rouge           0.000000   
50  Milliken / Agincourt North / Steeles East / L'...           0.000000   
51  Mimico NW / The Queensway West / South of Bloo...           0.000000   
52           

In [187]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)


In [182]:
kclusters = 5
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [ ]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [191]:
toronto_merged = dff
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_merged.head() # check the last columns!

Postal Code           Borough                                  Neighborhood  \
0         M3A        North York                                     Parkwoods   
1         M4A        North York                              Victoria Village   
2         M5A  Downtown Toronto                    Regent Park / Harbourfront   
3         M6A        North York             Lawrence Manor / Lawrence Heights   
4         M7A  Downtown Toronto  Queen's Park / Ontario Provincial Government   

    Latitude  Longitude  Cluster Labels 1st Most Common Venue  \
0  43.753259 -79.329656             0.0                  Park   
1  43.725882 -79.315572             1.0     French Restaurant   
2  43.654260 -79.360636             1.0           Coffee Shop   
3  43.718518 -79.464763             1.0        Clothing Store   
4  43.662301 -79.389494             1.0           Coffee Shop   

    2nd Most Common Venue 3rd Most Common Venue        4th Most Common Venue  \
0       Food & Drink Shop         Deli / Bodega  Eastern European Restaurant   
1             Coffee Shop          Hockey Arena        Portuguese Restaurant   
2                  Bakery                   Pub                         Park   
3  Furniture / Home Store     Accessories Store                     Boutique   
4        Sushi Restaurant                 Diner                  Yoga Studio   

  5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
0   Dumpling Restaurant             Drugstore            Donut Shop   
1             Drugstore   Dumpling Restaurant            Donut Shop   
2        Breakfast Spot            Restaurant                  Café   
3           Event Space            Shoe Store   Sporting Goods Shop   
4        Discount Store                  Bank                   Bar   

  8th Most Common Venue  9th Most Common Venue 10th Most Common Venue  
0      Doner Restaurant                Dog Run    Distribution Center  
1      Doner Restaurant                Dog Run           Dance Studio  
2               Theater             Beer Store             Shoe Store  
3    Miscellaneous Shop  Vietnamese Restaurant              Gift Shop  
4                  Café               Beer Bar             Hobby Shop

In [196]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        fill=True,
        fill_opacity=0.7).add_to(map_clusters)
       

In [197]:
map_clusters